In [1]:
import random

In [ ]:
with open('data/aoc-2021-23.txt') as f:
    dat = [x.strip() for x in f.readlines()]
row1 = [ord(x)-ord('A') for x in dat[2].split('#')[3:7]]
row2 = [ord(x)-ord('A') for x in dat[3].split('#')[1:5]]


In [16]:
class amphipod:
    last_pos = ('room', 0, 0)
    curr_pos = ('room', 0, 0)
    dest_room = 0
    weight = 1
    dist = 0
    max_depth = 1
    
    def __init__(self, pos, dest_room, max_depth = 1):
        self.last_pos = pos
        self.curr_pos = pos
        self.weight = 10**dest_room
        self.dest_room = dest_room
        self.max_depth = max_depth
        
    def __repr__(self):
        return (f'dst {self.dest_room} @ {self.curr_pos}')
    
    def energy(self):
        return self.weight * self.dist

    def move(self, pos):
        self.last_pos = self.curr_pos
        self.curr_pos = pos
        if self.curr_pos[0] == 'hall':
            self.dist += 1 + self.last_pos[2]
            if self.curr_pos[1] < 2*self.last_pos[1]:
                self.dist += 2*self.last_pos[1] - self.curr_pos[1]
            else:
                self.dist += self.curr_pos[1] - 2*self.last_pos[1]
        else:
            self.dist += 1 + self.curr_pos[2]
            if 2*self.curr_pos[1] < self.last_pos[1]:
                self.dist += self.last_pos[1] - 2*self.curr_pos[1]
            else:
                self.dist += 2*self.curr_pos[1] - self.last_pos[1]
    
    def valid_moves(self, pods):
        if self.curr_pos[0] == 'room' and self.last_pos[0] == 'hall':
            return []
        if self.curr_pos[0] == 'room':
            if self.curr_pos[2] >= 1 and any([p.curr_pos[0] == 'room' and p.curr_pos[1] == self.curr_pos[1] and p.curr_pos[2] < self.curr_pos[2] for p in pods]):
                return []
            left = [p.curr_pos[1] for p in pods if p.curr_pos[0] == 'hall' and p.curr_pos[1] <= 2*self.curr_pos[1] - 1]
            right = [p.curr_pos[1] for p in pods if p.curr_pos[0] == 'hall' and p.curr_pos[1] >= 2*self.curr_pos[1] + 1]
            if 2*self.curr_pos[1]-1 in left and 2*self.curr_pos[1]+1 in right:
                return []
            else:
                lpos = [('hall', x, 0) for x in range(-2 if len(left) == 0 else max(left)+1, 2*self.curr_pos[1]) if x not in [0,2,4,6]]
                rpos = [('hall', x, 0) for x in range(2*self.curr_pos[1]+1, 9 if len(right) == 0 else min(right)) if x not in [0,2,4,6]]
                return lpos + rpos
        else:
            occ = [p for p in pods if p.curr_pos[0] == 'room' and p.curr_pos[1] == self.dest_room]
            if any([p.dest_room != self.dest_room for p in occ]):
                return []
            if 2*self.dest_room < self.curr_pos[1]:
                if any([p.curr_pos[0] == 'hall' and p.curr_pos[1] in range(2*self.dest_room+1, self.curr_pos[1]) for p in pods]):
                    return []
            else:
                if any([p.curr_pos[0] == 'hall' and p.curr_pos[1] in range(self.curr_pos[1]+1, 2*self.dest_room) for p in pods]):
                    return []

            return [('room', self.dest_room, self.max_depth-len(occ))]

In [7]:
def random_game():
    pods = [amphipod(('room', i, 0), x) for i,x in enumerate(row1)] + [amphipod(('room', i, 1), x) for i,x in enumerate(row2)]

    moves = []
    
    while True:
        m = [(i, m) for i, p in enumerate(pods) for m in p.valid_moves(pods)]
        v = random.choice(m)
        pods[v[0]].move(v[1])
        moves += [v]
        if len(moves) > 50:
            return None
        if sum([len(p.valid_moves(pods)) for p in pods]) == 0:
            if all([p.curr_pos[0] == 'room' and p.curr_pos[1] == p.dest_room for p in pods]):
                return (sum([p.energy() for p in pods]), pods, moves)
            else:
                return None

In [15]:
res = [random_game() for i in range(10000)]
mv = min([r[0] for r in res if r is not None])
print(f'Part 1: {mv}')

Part 1: 15180
